In [58]:
import numpy as np 
import pandas as pd 
# import geopandas as gpd
from tqdm import tqdm
# from tensorflow.keras import models

data = pd.read_csv("train.csv")
data_size = len(data['Id'])
data["Country_Region"] = [country_name.replace("'","") for country_name in data["Country_Region"]]
data = data.query("Date>'2020-01-29' and Date<='2020-03-11'")
trend_df = pd.DataFrame(columns={"infection_trend","fatality_trend","expected_cases","expected_fatalities"})

# for country in data.Country_Region.unique():
#     print(country)

    # count = 0
    # for province in data.query("Country_Region=='"+country+"'").Province_State:
    #     count+=1

    # print(count)

with tqdm(total=len(data.Country_Region.unique())) as pbar:
    for country in data.Country_Region.unique():
        for province in data.query("Country_Region=='"+country+"'").Province_State:
            province_df = data.query("Country_Region=='"+country+"' and Province_State=='"+str(province)+"'")
            for i in range(0,len(province_df)-7,7):

                infection_trend = [float(x) for x in province_df[i:i+6].ConfirmedCases.values]
                fatality_trend = [float(x) for x in province_df[i:i+6].Fatalities.values]
                expected_cases = float(province_df.iloc[i+7].ConfirmedCases)
                expected_fatalities = float(province_df.iloc[i+7].Fatalities)
                                            
                trend_df = trend_df.append({"infection_trend":infection_trend,
                                 "fatality_trend":fatality_trend,
                                 "expected_cases":expected_cases,
                                 "expected_fatalities":expected_fatalities},ignore_index=True)
        pbar.update(1)

100%|██████████| 173/173 [02:27<00:00,  1.17it/s]


In [61]:
# print(trend_df)
trend_df["input"] = [np.asarray([np.array(trends["infection_trend"]),np.asarray(trends["fatality_trend"])]) for idx,trends in trend_df.iterrows()]
print(trend_df)

infection_trend  expected_cases  expected_fatalities  \
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
...                               ...             ...                  ...   
26245  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
26246  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
26247  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
26248  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   
26249  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0                  0.0   

                       fatality_trend  \
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
1  

In [76]:
from sklearn.utils import shuffle
trend_df = shuffle(trend_df)
trend_df.head()

,infection_trend,expected_cases,expected_fatalities,fatality_trend,input
15359,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
6160,"[278.0, 352.0, 422.0, 493.0, 566.0, 675.0]",851.0,2.0,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0]","[[278.0, 352.0, 422.0, 493.0, 566.0, 675.0], [..."
12766,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
18928,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
11944,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
